In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import math
import xlwt
from pathlib import Path

In [4]:

BASE_DIR = Path().absolute()
DIR = BASE_DIR/'Results'
DIR


WindowsPath('c:/Users/fan_shuyuan/Desktop/科研ing/工艺流程优化/ASBO-V6/Results')

In [5]:
*

SyntaxError: invalid syntax (796671574.py, line 1)

In [ ]:
sol = 'ASBO'
case_name = 'Case1'
runs_number = 100
list_realvalue = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return -obj,cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

for i in range(runs_number):
    optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=50, n_iter=10,p_hyper=[5, 2, 0.78, 0.95,1e-5],acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': -2.3269307027986507e-06, 'params': {'x1': -4.020306908294415, 'x2': -6.021089227928202}, 'constraint': array([-12.98047541,  -6.97870459, -75.37209556])}
{'target': -1.2414316320263125e-06, 'params': {'x1': -4.016204077101914, 'x2': -6.0081853533781056}, 'constraint': array([-12.9757772 ,  -6.99168336, -75.04848804])}
{'target': -1.094238398607315e-05, 'params': {'x1': -4.032980809247455, 'x2': -6.074143175498476}, 'constraint': array([-13.00818156,  -6.92531296, -76.69618677])}
{'target': -8.420453668672719e-07, 'params': {'x1': -3.991272449512709, 'x2': -6.021287561629923}, 'constraint': array([-13.03874266,  -6.97867435, -75.28389487])}
{'target': -5.61194072186243e-07, 'params': {'x1': -4.001868718965998, 'x2': -5.977839083442418}, 'constraint': array([-12.97410165,  -7.02215917, -74.27510081])}
{'target': -1.8200741352203502e-06, 'params': {'x1': -3.9797643940504295, 'x2': -6.000384355451591}, 'constraint': array([-13.04085557,  -6.9994109 , -74.74455278])}
{'target': -

In [ ]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case1'
runs_number = 100

def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3

initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-10,-15],ub=[5,15])
from itertools import product
for combination in product(x_random):
    recyc(combination)


C:\Users\admin\AppData\Local\Temp\ipykernel_103572\1513591222.py:26: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)


1.6217027975984465e-11
1.6203292898954544e-11
6.4275202588004e-13
4.0325997196072693e-10
1.6244808905159553e-11
1.635132877909816e-11
1.6234903154481338e-11
1.6119690436511297e-11
1.6157566058066195e-11
1.0041531741073865e-08
1.6189946882016815e-11
1.617088999025901e-11
1.6333206037892048e-11
1.0028544866299738e-08
`x0` violates bound constraints.
`x0` violates bound constraints.
1.612965470873235e-11
1.6219025538977222e-11
1.6347599884372957e-11
6.427312630519508e-13
1.6256694461224292e-11
1.643602037045508e-11
1.61621466968796e-11
1.6443081669478155e-11
6.423085769887464e-13
6.426791609865633e-13
1.112007547822193e-15
1.6372099845546343e-11
6.515702484973126e-13
1.627615866813582e-11
4.087121912825995e-10
1.6164174630811384e-11
0.00011904033992424599
2.5793425053136877e-14
1.6467818107286178e-11
1.6443180548601712e-11
1.632494043930055e-11
1.5987515976581866e-11
1.629223277573461e-11
2.557145337178978e-14
4.023315227525132e-10
4.0168560065626145e-10
1.6402136956937845e-11
1.609683733

In [ ]:
sol = 'slsqp'
case_name = 'Case1'
runs_number = 100
initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []


def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return obj,c1,c2,c3

def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-10,-15],ub=[5,15])
from itertools import product
for combination in product(x_random):
    recyc(combination)


4.471577426991125e-08
3.451557877822973e-19
1.2862269608902344e-12
3.306827080097318e-19
2.8571334173813692e-19
2.0319490422698994e-07
1.0561532108036624e-14
1.1987198089041464e-08
9.781114914426171e-16
2.803777664005695e-09
4.681462829406136e-07
2.41836777397759e-17
4.5581119930380193e-07
2.949111428202884e-09
3.279012188866105e-09
9.998211754796682e-20


C:\Users\admin\AppData\Local\Temp\ipykernel_103572\3374778901.py:25: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


1.9828825410103807e-09
2.6058424223081437e-19
4.5136730682198615e-07
2.447239802168654e-17
3.4717322209277504e-09
5.416951372749177e-18
1.7536453182358727e-08
7.281921471902146e-17
4.685392822519147e-07
3.7822601221538136e-17
4.4960887279679736e-17
1.5550823794286697e-11
4.915208640886543e-09
5.524697819714967e-15
5.230745948318769e-16
8.921146154128479e-19
9.092733807989284e-08
2.311546845672099e-16
2.2544419954004155e-07
9.891085771605067e-19
4.274788675860503e-17
4.502197402075233e-05
1.2208564877815253e-17
4.086606528844071e-11
1.4214557767225539e-08
3.475818388920501e-18
1.687677324777748e-09
1.845753302664673e-08
2.862366291380319e-08
3.4443926638790055e-07
7.811450078673667e-09
3.5532994457922185e-19
2.507194443051225e-09
3.345100772143352e-08
9.949494381041713e-08
9.745855630359627e-18
1.4562875306797154e-08
1.0634359134331383e-09
1.4691514735143212e-15
2.2654214616109106e-10
7.97034230259307e-09
1.9394006043617436e-08
7.819363246535978e-08
1.0296948250958179e-14
1.564209202156

In [ ]:
sol = 'CBO'
case_name = 'Case1'
list_realvalue = []
runs_number = 100
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

for i in range(runs_number):
    optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=50, n_iter=10,acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': -5.455644286023793e-07, 'params': {'x1': -4.010835022295992, 'x2': -6.004627867016621}, 'constraint': array([-12.98295782,  -6.99531343, -74.94580719])}
{'target': -1.367714610585897e-06, 'params': {'x1': -3.982615065874239, 'x2': -6.00469035300368}, 'constraint': array([-13.03946022,  -6.99515853, -74.85704113])}
{'target': -4.6471311903758557e-07, 'params': {'x1': -3.9907903318434697, 'x2': -5.991113501425182}, 'constraint': array([-13.00953284,  -7.00884409, -74.557428  ])}
{'target': -5.851094302716218e-08, 'params': {'x1': -4.003549014614085, 'x2': -5.9984907519088955}, 'constraint': array([-12.99139272,  -7.00150295, -74.77514197])}
{'target': -3.767704187111443e-06, 'params': {'x1': -3.971920826102246, 'x2': -6.015400446138962}, 'constraint': array([-13.07155879,  -6.98420533, -75.08038939])}
{'target': -2.2925365208824453e-06, 'params': {'x1': -4.01521060784939, 'x2': -5.966268227081836}, 'constraint': array([-12.93584701,  -7.03361609, -74.04143333])}
{'target': -6.

In [ ]:
sol = 'botorch'
case_name = 'Case1'
list_realvalue = []
iterations = []
max_values = []
runs_number = 100
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    itera=itera+1
    return -obj,c1,c2,c3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 50 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, -1e6, -1e6, ], dtype=torch.float64)
ub =  torch.tensor([0, 0,0 ], dtype=torch.float64)
bounds = torch.tensor([[-10, -15,], [5, 15]])

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    return target,c1,c2,c3
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], 
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []
                
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3],  
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C = torch.cat([C1, C2, C3], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    list_realvalue.append(max(Valid_Y[0]))
                if len(list_realvalue) == 10:
                    max_value = max(list_realvalue).item()
                    print("Valid_Y",max_value)
                    max_values.append(max_value)
                    f = xlwt.Workbook('encoding = utf-8')
                    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    for i in range(len(iterations)):
                        sheet1.write(i,0,max_values[i])
                        sheet1.write(i,1,iterations[i])
                    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
                    list_realvalue.clear()
                    break
        except Exception as e:
            max_value = max(list_realvalue).item()
            print("Attempting another calculation...",max_value)
            max_values.append(max_value)
            f = xlwt.Workbook('encoding = utf-8')
            sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            for i in range(len(iterations)):
                sheet1.write(i,0,max_values[i])
                sheet1.write(i,1,iterations[i])
            f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
            list_realvalue.clear()
run_multiple_calculations(num_runs=runs_number)


Valid_Y -0.09121137637097619
Valid_Y -0.021310174694204428
Valid_Y -0.5899518683652004
Valid_Y -0.22805580953021548
Valid_Y -0.016420109159744983
Valid_Y -0.03984088774218665
Valid_Y -0.12633641307071458
Valid_Y -0.1664372364899613
Valid_Y -0.03608009945860714
Valid_Y -0.022338516271449997
Valid_Y -0.02473233628312422
Valid_Y -0.012330227695538912
Valid_Y -0.04134240317759926
Valid_Y -0.07058500415497519
Valid_Y -0.2572755865710583
Valid_Y -0.001043250184602763
Valid_Y -0.015551271910762316
Valid_Y -0.051307125327001483
Valid_Y -0.009385796768607374
Valid_Y -0.562832925945328
Valid_Y -0.20802946246762774
Valid_Y -0.3733408493345734
Valid_Y -0.32336768946955197
Valid_Y -0.02298810075391273
Valid_Y -0.023107834748705088
Valid_Y -0.02461891994821532
Valid_Y -0.4534310102215059
Valid_Y -0.06566843735231273
Valid_Y -0.03922525718837519
Valid_Y -0.5049157743010488
Valid_Y -0.14689727979797026
Valid_Y -0.47955696910858087
Valid_Y -0.5334293551458535
Valid_Y -0.4403956529944324
Valid_Y -0.0386

In [8]:
sol = 'PSO'
case_name = 'Case1'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [-c1,-c2,-c3]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb=[-10,-15]
ub=[5,15]
num=0
for num in range(runs_number):
    xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=50, omega=0.5, phip=0.5, phig=0.5, maxiter=10, minstep=1e-8,minfunc=1e-8, debug=False)
    print('xopt, fopt',xopt, fopt)
    xopt = np.append(xopt,fopt)
    list_realvalue.extend([xopt])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,x_values[2])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.02750338 -5.97389362] 4.119207571395221e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.05461206 -5.88492518] 2.7969024244224856e-05
Stopping search: maximum iterations reached --> 10
xopt, fopt [-3.98083561 -6.013635  ] 1.8388989674686846e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.0363848  -6.02894796] 6.8148867215376e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-3.98523375 -5.98924243] 1.0976594167609744e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.00288929 -5.96376019] 1.4963508600799265e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-3.99388779 -5.96653964] 1.4100354723722786e-06
Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.00167887 -5.99766273] 1.859687106466827e-08
Stopping search: maximum iterations reached --> 10
xopt, fopt [-4.00495587 -6.00776093] 1.7608295825796585e-07
Stoppi

In [ ]:
sol = 'GA'
case_name = 'Case1'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    itera=itera+1
    return obj,c1,c2,c3

lb=[-10,-15]
ub=[5,15]
num=0
for num in range(runs_number):
    ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=50, max_iter=10, lb =[-10,-15],ub=[5,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1],lambda x:func(x)[2],lambda x:func(x)[3]])
    ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)q
    best_x, best_y = ga.run()
    best_res = np.concatenate((best_y, best_x))
    print('best_res',best_res)
    list_realvalue.extend([best_res])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,x_values[2])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


best_res [ 2.50673956e-04 -3.89925601e+00 -5.56987310e+00]
best_res [ 5.69661988e-05 -4.09388693e+00 -6.12653284e+00]
best_res [ 1.63583621e-05 -4.02149388e+00 -5.88653380e+00]
best_res [ 9.18487332e-05 -3.97717652e+00 -5.71613345e+00]
best_res [ 1.00750278e-04 -3.85429212e+00 -6.07584266e+00]
best_res [ 4.98728766e-03 -2.94455969e+00 -5.81905278e+00]
best_res [ 1.23322788e-05 -4.05267546e+00 -6.00048606e+00]
best_res [ 6.70210301e-05 -4.02350051e+00 -6.24105981e+00]
best_res [ 1.35552510e-03 -4.26598612e+00 -5.03202369e+00]
best_res [ 2.46540388e-04 -4.09955879e+00 -5.57310594e+00]
best_res [ 1.03933377e-04 -4.09691486e+00 -6.23657996e+00]
best_res [ 0.00481141 -3.94933614 -3.92153904]
best_res [ 4.93008468e-04 -3.91332453e+00 -6.64316178e+00]
best_res [ 1.65682331e-04 -3.96451968e+00 -5.62042302e+00]
best_res [ 1.03402298e-03 -4.15708184e+00 -5.08790249e+00]
best_res [ 2.10318774e-03 -3.86074949e+00 -4.65266703e+00]
best_res [ 5.13169637e-05 -3.94007912e+00 -5.82160937e+00]
best_res 